# Mathematical Formulation
**Decision Variables:**
$$
X_{ij} = \text{Square feet (in 1000s) leased at beginning of month} i \text{for} j \text{months}\\
$$

**Objective**

$$ \text{Minimize  } 	300 (X_{11} + X_{21} + X_{31} + X_{41} + X_{51}) + 525 (X_{12} + X_{22} + X_{32} + X_{42} ) + 775 (X_{13} + X_{23} + X_{33} )+ 850 (X_{14} + X_{24}) + 975 X_{15} 
$$
**Constraints**
$$\text{month 1: } X_{11} + X_{12} + X_{13} + X_{14} + X_{15} > 25\\
\text{month 2: } X_{21} + X_{22} + X_{23} + X_{24} + X_{12} + X_{13} + X_{14} + X_{15} > 10 \\
\text{month 3: } X_{31} + X_{32} + X_{33} + X_{22} + X_{23} + X_{24} + X_{13} + X_{14} + X_{15} > 20 \\
\text{month 4: }X_{41} + X_{42} + X_{32} + X_{33} + X_{23} + X_{24} + X_{14} + X_{15} > 10 \\
\text{month 5: } X_{51} + X_{42} + X_{33} + X_{24}  + X_{15} > 5 \\
			X_{ij} > 0
$$
<img src="Picture1.png">

In [4]:
from pulp import *
import pandas as pd

In [5]:
probA=LpProblem("Problem A",LpMinimize)
#Constraint and Decision Variable
Starting_month = [1,2,3,4,5]
Length_of_lease = [1,2,3,4,5]
Demand = [25,10,20,10,5]
Demand = dict(zip(Starting_month,Demand))
Cost = [300,525,775,850,975]
Cost = dict(zip(Length_of_lease, Cost))

#Define decision variables with the starting month and lease length. 
#Starting from month 1, we have leases with 5 length. From month 2, we have 4. ... From month 5, we have only 1. 

combination = [(i,j) for i in Starting_month for j in range(1,7-i)]

Space_lease_vars = LpVariable.dicts("x",combination,lowBound=0,cat='Continuous')
#for i in range(len(Starting_month)):
#    if i==0:
#        Space_lease_vars = LpVariable.dicts("x",(Starting_month[i:i+1],Length_of_lease[0:5-i]),lowBound=0,cat='Continuous')
#    else:
#        Space_lease_vars.update(LpVariable.dicts("x",(Starting_month[i:i+1],Length_of_lease[0:5-i]),lowBound=0,cat='Continuous'))

Space_lease_vars

{(1, 1): x_(1,_1),
 (1, 2): x_(1,_2),
 (1, 3): x_(1,_3),
 (1, 4): x_(1,_4),
 (1, 5): x_(1,_5),
 (2, 1): x_(2,_1),
 (2, 2): x_(2,_2),
 (2, 3): x_(2,_3),
 (2, 4): x_(2,_4),
 (3, 1): x_(3,_1),
 (3, 2): x_(3,_2),
 (3, 3): x_(3,_3),
 (4, 1): x_(4,_1),
 (4, 2): x_(4,_2),
 (5, 1): x_(5,_1)}

## Step 4: Define objective and constraints using += operator and lpSum
- **lpSum(vector)** calculates the sum of a list of linear expressions. 
- The parameter is a list of linear expressions

- **Constraints**: (Cover space requirement of each month)
- x_month1_1 + x_month1_2 + x_month1_3 + x_month1_4 + x_month1_5 >= 25
- x_month1_2 + x_month1_3 + x_month1_4 + x_month1_5 + x_month2_1 + x_month2_2 + x_month2_3 + x_month2_4 >= 10
- x_month1_3 + x_month1_4 + x_month1_5 + x_month2_2 + x_month2_3+ x_month2_4 + x_month3_1 + x_month3_2 + x_month3_3 >= 20
- x_month1_4 + x_month1_5 + x_month2_3 + x_month2_4 + x_month3_2 + x_month3_3 + x_month4_1 + x_month4_2 >= 10
- x_month1_5 + x_month2_4 + x_month3_3 + x_month4_2 + x_month5_1 >= 5



In [6]:
# Objective Function

probA+=lpSum(Space_lease_vars[(i,j)]*Cost[j] for j in Length_of_lease for i in range(1,7-j))
#Minimize the total cost of all the leases. 

#Demand constraint
for i in Starting_month:
    probA += lpSum(Space_lease_vars[(j,k)] for j in range(1,i+1) for k in range(i+1-j,7-j) ) >= Demand[i]
# range(1,i+1) describes how many months we need to consider in each starting month (all the months before). 
# range (i+1-j,7-j) describes what are the leases we need to look at.
#For each starting month i, we keep all the lease that start from this month and other leases that cover this month. 
#For example, in month 1, we keep all the lease starting from month 1. Which is: x_month1_1 + x_month1_2 + x_month1_3 + x_month1_4 + x_month1_5
#In month 2, we keep all the lease starting from month 2 and leases that start from month 1 but with length larger or equal than 2. 
#Which is: x_month1_2 + x_month1_3 + x_month1_4 + x_month1_5 + x_month2_1 + x_month2_2 + x_month2_3 + x_month2_4
#Month 3: x_month1_3 + x_month1_4 + x_month1_5 + x_month2_2 + x_month2_3 + x_month2_4 + x_month3_1 + x_month3_2 + x_month3_3


In [7]:
probA

Problem A:
MINIMIZE
300*x_(1,_1) + 525*x_(1,_2) + 775*x_(1,_3) + 850*x_(1,_4) + 975*x_(1,_5) + 300*x_(2,_1) + 525*x_(2,_2) + 775*x_(2,_3) + 850*x_(2,_4) + 300*x_(3,_1) + 525*x_(3,_2) + 775*x_(3,_3) + 300*x_(4,_1) + 525*x_(4,_2) + 300*x_(5,_1) + 0
SUBJECT TO
_C1: x_(1,_1) + x_(1,_2) + x_(1,_3) + x_(1,_4) + x_(1,_5) >= 25

_C2: x_(1,_2) + x_(1,_3) + x_(1,_4) + x_(1,_5) + x_(2,_1) + x_(2,_2)
 + x_(2,_3) + x_(2,_4) >= 10

_C3: x_(1,_3) + x_(1,_4) + x_(1,_5) + x_(2,_2) + x_(2,_3) + x_(2,_4)
 + x_(3,_1) + x_(3,_2) + x_(3,_3) >= 20

_C4: x_(1,_4) + x_(1,_5) + x_(2,_3) + x_(2,_4) + x_(3,_2) + x_(3,_3)
 + x_(4,_1) + x_(4,_2) >= 10

_C5: x_(1,_5) + x_(2,_4) + x_(3,_3) + x_(4,_2) + x_(5,_1) >= 5

VARIABLES
x_(1,_1) Continuous
x_(1,_2) Continuous
x_(1,_3) Continuous
x_(1,_4) Continuous
x_(1,_5) Continuous
x_(2,_1) Continuous
x_(2,_2) Continuous
x_(2,_3) Continuous
x_(2,_4) Continuous
x_(3,_1) Continuous
x_(3,_2) Continuous
x_(3,_3) Continuous
x_(4,_1) Continuous
x_(4,_2) Continuous
x_(5,_1) Contin

## Step 5: Run solver

In [8]:
#probA.writeLP("Space_lease.lp") #optional
probA.solve()
print("Status:",LpStatus[probA.status])

Status: Optimal


## Step 6: Format PuLP Solution Output

In [10]:
print("Total cost=", value(probA.objective))
output=[]

for i in Starting_month:
    var_output=[]
    for k in Length_of_lease[0:6-i]:
        var_output.append(Space_lease_vars[(i,k)].varValue)
        
    output.append(var_output)
    
print(output)
output_df = pd.DataFrame(output,index=Starting_month,columns=Length_of_lease)
output_df

Total cost= 16625.0
[[15.0, 0.0, 0.0, 5.0, 5.0], [0.0, 0.0, 0.0, 0.0], [10.0, 0.0, 0.0], [0.0, 0.0], [0.0]]


,1,2,3,4,5
1,15.0,0.0,0.0,5.0,5.0
2,0.0,0.0,0.0,0.0,NaN
3,10.0,0.0,0.0,NaN,NaN
4,0.0,0.0,NaN,NaN,NaN
5,0.0,NaN,NaN,NaN,NaN
